In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
from openai import OpenAI
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib
import matplotlib.pyplot as plt
import pickle

In [3]:
openai_api_key = ''

In [4]:
client = OpenAI(api_key= openai_api_key)
articles_df = pd.DataFrame(pd.read_csv('../data/Result_9.csv')) 
articles_df

,title,tags
0,한국에서 돈을 가장 빨리 불리는 비법 (feat.13월의 월급 두둑해짐),"재테크,etf,연금저축,투자,주식,미국주식,비법,꿀팁,돈불리기"
1,대출상품의 맹점을 활용해 저금리로 대출받아 서울에 내집 마련하기!,"재테크,투자,부동산"
2,애들건강 음식\nㅕ,"삶은고기,채소,반나나,요플레,애호박"
3,운동\n,테그
4,[웹소설 전망] 앞으로 웹소설을 혼자 써서 성공하기는 힘들 겁니다,"성공비결,웹소설,성공공식,집단창작,공동창작,콘텐츠,전망"
5,웹소설로 돈 벌려면 이 3가지를 익히셔야 합니다,"성공공식,세컨드잡,N잡,N잡러,웹소설,인기작품,성공비결"
6,"웹소설, 같이 쓰면 얼마나 더 나아지는지 보여드림","부업,웹소설,성공공식,N잡러,인기작품,N잡,성공비결"
7,웹소설을 같이 쓴다는 것,"1등,웹소설,카카오페이지"
8,카카오페이지 웹소설 1위 찍다!,"웹소설,인기작품,카카오페이지,1등"
9,발로란트,"세이지,로터스,게임,발로란트"


In [5]:
def get_embedding(text: str, model="text-embedding-3-small"):
    text = text.replace('\n', ' ')
    return client.embeddings.create(input=[text], model=model).data[0].embedding

articles_df['embedding'] = articles_df.apply(lambda x: get_embedding(f"{x['title']} - {' '.join(x['tags'].split(','))}" ), axis=1)

with open( "embedded-article", "wb" ) as file:
    pickle.dump( articles_df, file)

articles_df
    

,title,tags,embedding
0,한국에서 돈을 가장 빨리 불리는 비법 (feat.13월의 월급 두둑해짐),"재테크,etf,연금저축,투자,주식,미국주식,비법,꿀팁,돈불리기","[0.018516482785344124, 0.022310104221105576, -..."
1,대출상품의 맹점을 활용해 저금리로 대출받아 서울에 내집 마련하기!,"재테크,투자,부동산","[0.008505742996931076, -0.0037927422672510147,..."
2,애들건강 음식\nㅕ,"삶은고기,채소,반나나,요플레,애호박","[0.035317935049533844, -0.02704542875289917, -..."
3,운동\n,테그,"[-0.031634896993637085, 0.02379104495048523, -..."
4,[웹소설 전망] 앞으로 웹소설을 혼자 써서 성공하기는 힘들 겁니다,"성공비결,웹소설,성공공식,집단창작,공동창작,콘텐츠,전망","[0.06743671000003815, 0.009778322651982307, 0...."
5,웹소설로 돈 벌려면 이 3가지를 익히셔야 합니다,"성공공식,세컨드잡,N잡,N잡러,웹소설,인기작품,성공비결","[0.027522113174200058, 0.008336253464221954, -..."
6,"웹소설, 같이 쓰면 얼마나 더 나아지는지 보여드림","부업,웹소설,성공공식,N잡러,인기작품,N잡,성공비결","[0.030167724937200546, 0.007432451471686363, -..."
7,웹소설을 같이 쓴다는 것,"1등,웹소설,카카오페이지","[0.012308107689023018, 0.004747177939862013, 0..."
8,카카오페이지 웹소설 1위 찍다!,"웹소설,인기작품,카카오페이지,1등","[0.0616585947573185, -0.0006531630642712116, 0..."
9,발로란트,"세이지,로터스,게임,발로란트","[0.005846256855875254, 0.012760835699737072, -..."


In [21]:
query_test_description = ("소설 써보고 싶은데 꿀팁 있을까?")
query_embedding = get_embedding(query_test_description)

articles_df['similarity'] = articles_df['embedding'].apply(lambda x: cosine_similarity(np.array(x).reshape(1, -1), np.array(query_embedding).reshape(1, -1))[0][0])
selected_df = articles_df.sort_values(by='similarity', ascending=False)
selected_df = selected_df[selected_df['similarity'] > 0.3]
selected_df

,title,tags,embedding,similarity
4,[웹소설 전망] 앞으로 웹소설을 혼자 써서 성공하기는 힘들 겁니다,"성공비결,웹소설,성공공식,집단창작,공동창작,콘텐츠,전망","[0.06743671000003815, 0.009778322651982307, 0....",0.348265
7,웹소설을 같이 쓴다는 것,"1등,웹소설,카카오페이지","[0.012308107689023018, 0.004747177939862013, 0...",0.316342
6,"웹소설, 같이 쓰면 얼마나 더 나아지는지 보여드림","부업,웹소설,성공공식,N잡러,인기작품,N잡,성공비결","[0.030167724937200546, 0.007432451471686363, -...",0.312039
5,웹소설로 돈 벌려면 이 3가지를 익히셔야 합니다,"성공공식,세컨드잡,N잡,N잡러,웹소설,인기작품,성공비결","[0.027522113174200058, 0.008336253464221954, -...",0.312034
